In [1]:
import tensorflow as tf
from tensorflow import keras

### Creating the model

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [3]:
input_shape = (150, 150, 3)
inputs = keras.Input(shape=input_shape)

In [4]:
conv_layer = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)

In [5]:
pool_size = (2, 2)
pool_layer = MaxPooling2D(pool_size=pool_size)(conv_layer)

In [6]:
flatten_layer = Flatten()(pool_layer)

In [7]:
dense_layer1 = Dense(64, activation="relu")(flatten_layer)
outputs = Dense(1, activation="sigmoid")(dense_layer1)

In [8]:
model = keras.Model(inputs, outputs)

#### Question 1

BinaryCrossentropy

In [9]:
from keras.losses import BinaryCrossentropy

In [10]:
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.002, momentum=0.8),
    loss=BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

#### Question 2

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
train_gen = ImageDataGenerator(rescale=1./255)
train_ds = train_gen.flow_from_directory(
    "./train",
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1./255)
val_ds = val_gen.flow_from_directory(
    "./validation",
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [14]:
history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)
history.history

Epoch 1/10
100/100 [==============================] - 14s 139ms/step - loss: 0.6975 - accuracy: 0.5320 - val_loss: 0.6857 - val_accuracy: 0.5330
Epoch 2/10
100/100 [==============================] - 15s 149ms/step - loss: 0.6879 - accuracy: 0.5385 - val_loss: 0.6815 - val_accuracy: 0.5480
Epoch 3/10
100/100 [==============================] - 15s 147ms/step - loss: 0.6844 - accuracy: 0.5415 - val_loss: 0.6929 - val_accuracy: 0.5130
Epoch 4/10
100/100 [==============================] - 14s 144ms/step - loss: 0.6814 - accuracy: 0.5650 - val_loss: 0.6770 - val_accuracy: 0.6000
Epoch 5/10
100/100 [==============================] - 15s 148ms/step - loss: 0.6658 - accuracy: 0.5885 - val_loss: 0.6720 - val_accuracy: 0.5800
Epoch 6/10
100/100 [==============================] - 15s 153ms/step - loss: 0.6731 - accuracy: 0.5990 - val_loss: 0.6662 - val_accuracy: 0.5900
Epoch 7/10
100/100 [==============================] - 16s 159ms/step - loss: 0.6668 - accuracy: 0.5945 - val_loss: 0.6683 - val_ac

{'loss': [0.6975035667419434,
  0.687885582447052,
  0.6844290494918823,
  0.6813533902168274,
  0.6657937169075012,
  0.6731157302856445,
  0.6668072938919067,
  0.6654340028762817,
  0.6617560982704163,
  0.6459524035453796],
 'accuracy': [0.5320000052452087,
  0.5385000109672546,
  0.5414999723434448,
  0.5649999976158142,
  0.5885000228881836,
  0.5989999771118164,
  0.5945000052452087,
  0.6010000109672546,
  0.5979999899864197,
  0.6330000162124634],
 'val_loss': [0.6856961250305176,
  0.6815395951271057,
  0.6929354071617126,
  0.6770449876785278,
  0.6720036864280701,
  0.6662291288375854,
  0.6682980060577393,
  0.6705685257911682,
  0.6483723521232605,
  0.6495100259780884],
 'val_accuracy': [0.5329999923706055,
  0.5479999780654907,
  0.5130000114440918,
  0.6000000238418579,
  0.5799999833106995,
  0.5899999737739563,
  0.6000000238418579,
  0.5770000219345093,
  0.6230000257492065,
  0.6159999966621399]}

#### Question 3

In [15]:
import statistics

In [16]:
statistics.mean(history.history['accuracy'])

0.5791000008583069

#### Question 4

In [17]:
statistics.stdev(history.history['loss'])

0.015010896724238278

### Data Augmentation

In [18]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_ds = train_gen.flow_from_directory(
    "./train",
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)


Found 20000 images belonging to 2 classes.


In [19]:
history2 = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)
history2.history

Epoch 1/10
100/100 [==============================] - 26s 264ms/step - loss: 0.6750 - accuracy: 0.5680 - val_loss: 0.6665 - val_accuracy: 0.5810
Epoch 2/10
100/100 [==============================] - 25s 254ms/step - loss: 0.6761 - accuracy: 0.5645 - val_loss: 0.6524 - val_accuracy: 0.6050
Epoch 3/10
100/100 [==============================] - 27s 267ms/step - loss: 0.6726 - accuracy: 0.5765 - val_loss: 0.6563 - val_accuracy: 0.6240
Epoch 4/10
100/100 [==============================] - 26s 262ms/step - loss: 0.6696 - accuracy: 0.5835 - val_loss: 0.6494 - val_accuracy: 0.6280
Epoch 5/10
100/100 [==============================] - 26s 259ms/step - loss: 0.6754 - accuracy: 0.5735 - val_loss: 0.6535 - val_accuracy: 0.6340
Epoch 6/10
100/100 [==============================] - 28s 277ms/step - loss: 0.6568 - accuracy: 0.6005 - val_loss: 0.6413 - val_accuracy: 0.6280
Epoch 7/10
100/100 [==============================] - 27s 267ms/step - loss: 0.6675 - accuracy: 0.5820 - val_loss: 0.6430 - val_ac

{'loss': [0.6749874353408813,
  0.6760843992233276,
  0.6726104617118835,
  0.6695953607559204,
  0.6753665804862976,
  0.6568015813827515,
  0.6675153374671936,
  0.6630427241325378,
  0.6657806038856506,
  0.6646631956100464],
 'accuracy': [0.5680000185966492,
  0.5644999742507935,
  0.5764999985694885,
  0.5835000276565552,
  0.5734999775886536,
  0.6004999876022339,
  0.5820000171661377,
  0.5985000133514404,
  0.5899999737739563,
  0.5864999890327454],
 'val_loss': [0.6665012836456299,
  0.6523531675338745,
  0.6562963128089905,
  0.6494044661521912,
  0.6534850001335144,
  0.6412724852561951,
  0.6429857611656189,
  0.6295411586761475,
  0.6449108719825745,
  0.6338860392570496],
 'val_accuracy': [0.5809999704360962,
  0.6050000190734863,
  0.6240000128746033,
  0.628000020980835,
  0.6340000033378601,
  0.628000020980835,
  0.6259999871253967,
  0.652999997138977,
  0.6209999918937683,
  0.6380000114440918]}

#### Question 5

In [20]:
statistics.mean(history2.history['val_loss'])

0.6470636546611785

#### Question 6

In [21]:
statistics.mean(history2.history['val_accuracy'][5:10])

0.6332000017166137